In [1]:
# !pip install openpyxl

In [2]:
import pandas as pd

df = pd.read_excel(r"C:\Users\jites\Desktop\Project_folder\gpt_dt_6000_2.xlsx")
print(df.shape)
df.head(3)

(6000, 5)


,ChiefComplaint,Symptoms,LabTest,Medicines,Paragraph
0,A young woman presented with Headache and dizz...,"Headache and dizziness and Palpitations, Unexp...","Thyroid function test, Vitamin B12 level, Pace...","Albuterol inhaler, Prednisone 20 mg, Neorof 1 ...",A young woman presented with Headache and dizz...
1,The patient mentioned having some problems Ski...,"Skin rash, Stiffness and swelling in the joint...","Ophthalmologic exam, MRI of the brain","Hydrocortisone cream, Cetirizine 10 mg",The patient mentioned having some problems Ski...
2,A patient came in today with complaints of Sho...,"Shortness of breath and Diarrhea, Painful swal...","Holter monitor, Echocardiogram, Fluid amylase,...","Albuterol inhaler, Prednisone 20 mg, Bruvam l ...",A patient came in today with complaints of Sho...


In [3]:
def separate_input_output(df):
    texts = df["Paragraph"].tolist()
    labels = []
    for row in df.itertuples(index=False):
        output = ' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])])
        labels.append(output)
    return texts, labels

inputs, targets = separate_input_output(df)



# from datasets import Dataset
# def create_dataset(df):
#     texts = df['Paragraph'].tolist()
#     labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]
    
#     # Create a list of dictionaries
#     data = [{'text': f"{text} => {label}"} for text, label in zip(texts, labels)]
#     return Dataset.from_list(data)

# inputs, targets = create_dataset(df)

In [4]:
print(inputs[4])
print()
print(targets[4])

Patient having problems Back pain and Chest pain suffering from this from last 15 days. The patient noted the following symptoms High fever, chills, and muscle pain, sunken eyes, lumps and knot-like pimples, high body temperature. After discussing medical history and current symptoms, it seems likely that she is experiencing early signs of rheumatoid arthritis. I recommended Complete blood count (CBC), Blood culture to confirm the diagnosis. Levothyroxine 50 mcg, Cyanocobalamin 1000 mcg, Volitra plus spray,  megasty 160mg,  benitowa 8mg 15 tabs packing take these medicines. The patient will follow up in a week for reassessment.

ChiefComplaint: Patient having problems Back pain and Chest pain suffering from this from last 15 days. The patient noted the following symptoms High fever, chills, and muscle pain, sunken eyes, lumps and knot-like pimples, high body temperature. After discussing medical history and current symptoms, it seems likely that she is experiencing early signs of rheum

In [5]:
print(inputs[820])
print()
print(targets[820])

The patient is experiencing Fatigue and weakness and Fatigue and weakness which has been ongoing for the past week. They have been dealing with Frequent loose stools, mild stomach cramping, arrhythmia, skin peeling, bumpy areas on skin and itchy. After discussing medical history and current symptoms, it seems likely that she is experiencing early signs of rheumatoid arthritis. I have referred her to do MRI of the lower back, Blood test for calcium, Cath lab charge,  ct limb,  culture + aerobic + vaginal swab includes antibiotic sensitivity,   these lab tests. The patient was prescribed Naproxen 250 mg, Methotrexate 10 mg to manage the condition. The patient will follow up in a week for reassessment.

ChiefComplaint: The patient is experiencing Fatigue and weakness and Fatigue and weakness which has been ongoing for the past week. They have been dealing with Frequent loose stools, mild stomach cramping, arrhythmia, skin peeling, bumpy areas on skin and itchy. After discussing medical hi

In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from transformers.optimization import get_scheduler
from transformers import EarlyStoppingCallback
from torch.optim import AdamW



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load BART tokenizer and model
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
model.to(device)

c:\Users\jites\anaconda3\envs\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jites\anaconda3\envs\gpu_env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [7]:
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    inputs, targets, test_size=0.10, random_state=42
)

# Create Dataset objects
train_dataset = Dataset.from_dict({'input_text': train_texts, 'target_text': train_labels})
val_dataset = Dataset.from_dict({'input_text': val_texts, 'target_text': val_labels})

In [8]:
print(train_dataset.shape)
print(val_dataset.shape)

(5400, 2)
(600, 2)


In [9]:
print(train_dataset['input_text'][40])
print(train_dataset['target_text'][40])

The patient presents with a chief complaint of Insomnia. Painful swallowing and red throat, itchy rash, moods swing these types of symptoms the patient is experiencing. I have referred her to do Complete blood count (CBC), Blood culture, ( bedside ) dvt scan,   these lab tests. The patient has been prescribed Levothyroxine 50 mcg, MultivitaminThe patient has been prescribed Levothyroxine 50 mcg, Multivitamin for symptomatic relief.Based on the findings, Levothyroxine 50 mcg, Multivitamin were prescribed.
ChiefComplaint: The patient presents with a chief complaint of Insomnia. Painful swallowing and red throat, itchy rash, moods swing these types of symptoms the patient is experiencing. Symptoms: Insomnia, Painful swallowing and red throat, itchy rash, moods swing LabTest: Complete blood count (CBC), Blood culture, ( bedside ) dvt scan,   Medicines: Levothyroxine 50 mcg, Multivitamin


In [10]:
# Preprocess the data
def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=256)
    targets = tokenizer(examples['target_text'], truncation=True, padding='max_length', max_length=256)
    inputs['labels'] = targets['input_ids']
    return inputs

# Map preprocess function to the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 600/600 [00:00<00:00, 969.14 examples/s]


In [11]:
print(train_dataset['input_text'][0])
print(train_dataset['target_text'][0])

Today I examined a patient who presented with Ear pain, Weight loss, Back pain suffering from this from last 15 days. The symptoms are as follows Severe cramping in the lower abdomen. From the discussion i came to know medical history and current symptoms of patient. I decided to order Rheumatoid factor test, Joint X-ray to investigate further. The patient was prescribed Ibuprofen 400 mg, Cyclobenzaprine 10 mg, Xq 10 capsules,  forcef 750mg injection,  crina ncr 15mg tablet to manage the condition. The patient will follow up in a week for reassessment.
ChiefComplaint: Today I examined a patient who presented with Ear pain, Weight loss, Back pain suffering from this from last 15 days. The symptoms are as follows Severe cramping in the lower abdomen. From the discussion i came to know medical history and current symptoms of patient. Symptoms: Ear pain, Weight loss, Back pain, Severe cramping in the lower abdomen LabTest: Rheumatoid factor test, Joint X-ray Medicines: Ibuprofen 400 mg, Cy

In [12]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-5,                       # Learning rate
    per_device_train_batch_size=8,            # Batch size for training
    per_device_eval_batch_size=8,             # Batch size for evaluation
    num_train_epochs=5,                       # Number of epochs
    weight_decay=0.01,                        # Weight decay
    warmup_steps=300,                         # Warmup steps
    gradient_accumulation_steps=4,            # Gradient accumulation
    optim="adamw_torch",                      # Optimizer (AdamW)
    save_strategy="epoch",                    # Save every epoch
    logging_dir="./logs",                     # Log directory
    overwrite_output_dir=True,
    eval_steps= 200, ## 400,
    save_steps= 300, ## 500,
    logging_steps= 100,
    load_best_model_at_end=True,  # Load best model at the end based on evaluation
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    save_total_limit=3,                    # Limit saved models
    eval_accumulation_steps=4,             # Accumulate eval gradients
)



optimizer = AdamW(model.parameters(), lr= 0.0002,
                weight_decay= 0.01)

num_training_steps = len(train_dataset) * training_args.num_train_epochs // training_args.per_device_train_batch_size

scheduler = get_scheduler("linear", optimizer=optimizer, 
                            num_warmup_steps=300, num_training_steps=num_training_steps)


# Trainer for fine-tuning the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )


# Resume training
trainer.train()

# Save model and tokenizer
model.save_pretrained('./bart-diagnosis-model')
tokenizer.save_pretrained('./bart-diagnosis-tokenizer')

  0%|          | 0/840 [00:00<?, ?it/s]c:\Users\jites\anaconda3\envs\gpu_env\Lib\site-packages\transformers\models\bart\modeling_bart.py:497: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 12%|█▏        | 100/840 [14:18<1:45:11,  8.53s/it]

{'loss': 3.5411, 'grad_norm': 2.664801836013794, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.59}


                                                   
 20%|██        | 168/840 [24:27<1:35:13,  8.50s/it]c:\Users\jites\anaconda3\envs\gpu_env\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'eval_loss': 0.000545397459063679, 'eval_runtime': 23.0425, 'eval_samples_per_second': 26.039, 'eval_steps_per_second': 3.255, 'epoch': 1.0}


 24%|██▍       | 200/840 [30:52<2:10:37, 12.25s/it]

...

                                                   
100%|██████████| 840/840 [3:11:08<00:00, 19.63s/it]

{'eval_loss': 0.00020752263662870973, 'eval_runtime': 17.4366, 'eval_samples_per_second': 34.41, 'eval_steps_per_second': 4.301, 'epoch': 4.98}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 840/840 [3:11:14<00:00, 13.66s/it]


{'train_runtime': 11473.993, 'train_samples_per_second': 2.353, 'train_steps_per_second': 0.073, 'train_loss': 0.4264305555838205, 'epoch': 4.98}


('./bart-diagnosis-tokenizer\\tokenizer_config.json',
 './bart-diagnosis-tokenizer\\special_tokens_map.json',
 './bart-diagnosis-tokenizer\\vocab.json',
 './bart-diagnosis-tokenizer\\merges.txt',
 './bart-diagnosis-tokenizer\\added_tokens.json')

In [3]:
# from transformers import BartTokenizer, BartForConditionalGeneration
# tokenizer = BartTokenizer.from_pretrained('./bart-diagnosis-tokenizer')
# model = BartForConditionalGeneration.from_pretrained('./bart-diagnosis-model')

from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained('./t5-segregate-tokenizer-20sept')
model = T5ForConditionalGeneration.from_pretrained('./t5-segregate-model-20sept')

def generate_output(paragraph):
    inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, padding='max_length', max_length=256)
    # print(inputs)
    # outputs = model.generate(**inputs, max_length=32, num_beams=4, early_stopping=True)
    outputs = model.generate(**inputs, max_new_tokens=256, num_beams=4, early_stopping=True)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()

    return labeled_sentences

# result = generate_output("The patient presents with a chief complaint of Skin rash may be this disease comes from friends or family. Symptoms reported include Feeling of nausea, with occasional vomiting. After discussing patient medical history and current symptoms, it seems likely that patient is experiencing this problem a long ago. Ultrasound of the abdomen, Blood test for amylase, Differential leucocyte count (dlc),   these investigation test you have to do. For treatment, Prednisolone 10 mg, Topiramate 50 mg were given. The patient will follow up in a week for reassessment.")
# result = generate_output("The patient presents with a history of intermittent palpitations and dizziness. A 24-hour Holter monitor is recommended to assess for any arrhythmias. Initial management includes a low-dose beta-blocker to control symptoms.")
result = generate_output("The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional dizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar episodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), chest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to rule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin sublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was scheduled for further evaluation based on investigation results.")
# result = generate_output("A 30-year-old female presents with fatigue, muscle aches, and low-grade fever for the past five days. She reports a recent history of upper respiratory infection two weeks prior, which has since resolved. On examination, her throat is mildly inflamed, and her lymph nodes are slightly enlarged. No signs of pneumonia on chest auscultation. Given the symptoms, I suspect she may be experiencing a viral syndrome or post-viral fatigue. I recommended ibuprofen for muscle pain, hydration, and rest. To rule out mononucleosis or thyroid dysfunction, I've ordered a complete blood count (CBC) and thyroid function tests. She should return if symptoms persist beyond two weeks.")
# result = generate_output("The patient, a 45-year-old male, presents with a chief complaint of intermittent chest pain for the past week. He describes the pain as a pressure-like sensation, radiating to the left arm, particularly during physical exertion. The patient has a history of hypertension, which he is managing with amlodipine. No other major comorbidities are reported. Upon examination, his vitals are stable, but an ECG reveals minor ST segment changes. My initial diagnosis is suspected angina. I've prescribed nitroglycerin sublingually for acute episodes and referred him for a stress test and lipid profile to further assess his cardiovascular risk. Follow-up in one week.")
# result = generate_output("The patient is a 60-year-old male with a long-standing history of Type 2 diabetes who presents with a complaint of frequent urination and increased thirst over the past two weeks. He is currently on metformin but has been inconsistent with diet and exercise. On physical exam, his BMI is 32, and his blood glucose reading is 250 mg/dL. Based on the clinical presentation, uncontrolled diabetes is likely. I've recommended adjustments to his medication regimen by adding a DPP-4 inhibitor and advised him to monitor his blood sugar more frequently. Additionally, I ordered an HbA1c test to evaluate long-term glucose control and referred him to a nutritionist for dietary counseling. Follow-up in two weeks.")

final_result = parse_output(result)
final_result

{'ChiefComplaint': ' The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional dizziness.',
 'Symptoms': ' intermittent, sharp chest pain on the left side, radiating to the left arm,',
 'LabTest': ' electrocardiogram (ECG), chest X-ray, and complete blood count (CBC)',
 'Medicines': ' Nitroglycerin sublingual tablets'}